## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Janakpur,NP,26.7183,85.9065,66.18,40,0,6.55,clear sky
1,1,Lorengau,PG,-2.0226,147.2712,81.54,74,95,1.90,overcast clouds
2,2,Kyaukse,MM,21.6000,96.1333,78.66,37,10,4.23,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,49.66,62,0,14.97,clear sky
4,4,Bluff,NZ,-46.6000,168.3333,62.80,69,100,10.98,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Janakpur,NP,26.7183,85.9065,66.18,40,0,6.55,clear sky
1,1,Lorengau,PG,-2.0226,147.2712,81.54,74,95,1.90,overcast clouds
2,2,Kyaukse,MM,21.6000,96.1333,78.66,37,10,4.23,clear sky
5,5,Busselton,AU,-33.6500,115.3333,70.61,52,66,9.89,broken clouds
10,10,Pacific Grove,US,36.6177,-121.9166,68.54,50,0,9.22,clear sky
11,11,Half Moon Bay,US,37.4636,-122.4286,70.02,25,0,11.50,clear sky
13,13,Ahipara,NZ,-35.1667,173.1667,71.73,66,42,9.64,scattered clouds
14,14,Avarua,CK,-21.2078,-159.7750,82.45,83,75,9.22,broken clouds
16,16,Kaitangata,NZ,-46.2817,169.8464,66.92,59,64,6.24,broken clouds
19,19,Cape Town,ZA,-33.9258,18.4232,69.12,69,0,9.22,clear sky


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 304 entries, 0 to 674
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              304 non-null    int64  
 1   City                 304 non-null    object 
 2   Country              300 non-null    object 
 3   Lat                  304 non-null    float64
 4   Lng                  304 non-null    float64
 5   Max Temp             304 non-null    float64
 6   Humidity             304 non-null    int64  
 7   Cloudiness           304 non-null    int64  
 8   Wind Speed           304 non-null    float64
 9   Current Description  304 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 26.1+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 673
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              300 non-null    int64  
 1   City                 300 non-null    object 
 2   Country              300 non-null    object 
 3   Lat                  300 non-null    float64
 4   Lng                  300 non-null    float64
 5   Max Temp             300 non-null    float64
 6   Humidity             300 non-null    int64  
 7   Cloudiness           300 non-null    int64  
 8   Wind Speed           300 non-null    float64
 9   Current Description  300 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 25.8+ KB


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Janakpur,NP,66.18,clear sky,26.7183,85.9065,
1,Lorengau,PG,81.54,overcast clouds,-2.0226,147.2712,
2,Kyaukse,MM,78.66,clear sky,21.6000,96.1333,
5,Busselton,AU,70.61,broken clouds,-33.6500,115.3333,
10,Pacific Grove,US,68.54,clear sky,36.6177,-121.9166,
11,Half Moon Bay,US,70.02,clear sky,37.4636,-122.4286,
13,Ahipara,NZ,71.73,scattered clouds,-35.1667,173.1667,
14,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,
16,Kaitangata,NZ,66.92,broken clouds,-46.2817,169.8464,
19,Cape Town,ZA,69.12,clear sky,-33.9258,18.4232,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()


,index,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Janakpur,NP,66.18,clear sky,26.7183,85.9065,Hotel Sita Palace
1,1,Lorengau,PG,81.54,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
2,2,Kyaukse,MM,78.66,clear sky,21.6000,96.1333,Shwe Thit Sar Guesthouse
3,5,Busselton,AU,70.61,broken clouds,-33.6500,115.3333,Observatory Guest House
4,10,Pacific Grove,US,68.54,clear sky,36.6177,-121.9166,Lovers Point Inn
...,...,...,...,...,...,...,...,...
295,665,Upington,ZA,81.23,broken clouds,-28.4478,21.2561,River Place Manor
296,667,Chowchilla,US,71.02,clear sky,37.1230,-120.2602,Days Inn by Wyndham Chowchilla Gateway to Yose...
297,668,Ewa Beach,US,84.15,clear sky,21.3156,-158.0072,Base Lodging
298,671,Salvador,SV,82.06,few clouds,13.8333,-88.9167,Chico's Ranch


In [24]:
hotel_df =  hotel_df.loc[hotel_df['Hotel Name']!='NaN']

hotel_df = hotel_df.reset_index()

In [25]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [27]:
locations

,Lat,Lng
0,26.7183,85.9065
1,-2.0226,147.2712
2,21.6000,96.1333
3,-33.6500,115.3333
4,36.6177,-121.9166
...,...,...
272,-8.2578,-49.2647
273,-28.4478,21.2561
274,37.1230,-120.2602
275,21.3156,-158.0072


In [28]:
# 11a. Add a marker layer for each city to the map. 

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
    'zoom_level':1.5
}
fig = gmaps.figure(layout=figure_layout)

# Access maps with unique API key
gmaps.configure(api_key=g_key)
locations = hotel_df[["Lat", "Lng"]]

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)


# 11b. Display the figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…